<a href="https://colab.research.google.com/github/Illumaria/iti-funds-nav-parser/blob/master/iti-funds-nav-parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [0]:
url = 'https://www.londonstockexchange.com/stock/RUSE/iti-funds-ucits-etf-sicav/analysis?page=' # initial link (unspecified page)

In [0]:
resp = requests.get(url + '1')

In [0]:
bs = BeautifulSoup(resp.text)

In [0]:
# Get the number of pages:
pages = bs.find('a', class_="page-number page-last").get('href')
num_pages = int(pages.split(sep='=')[1])

In [0]:
# Find all the partial links on the first page:
scripts_rusb = []
scripts_ruse = []
for tag in bs.find_all('a'):
    if 'EUROBOND: Net Asset Value' in tag.text:
        scripts_rusb.append(tag.get('href'))
    if 'ETF SICAV: Net Asset Value' in tag.text:
        scripts_ruse.append(tag.get('href'))

In [0]:
# Do the same for other pages:
for i in range(2,num_pages+1):
    resp = requests.get('{}{}'.format(url, i))
    bs = BeautifulSoup(resp.text)
    for tag in bs.find_all('a'):
        if 'EUROBOND: Net Asset Value' in tag.text:
            scripts_rusb.append(tag.get('href'))
        if 'ETF SICAV: Net Asset Value' in tag.text:
            scripts_ruse.append(tag.get('href'))

In [0]:
# Get full links:
urls_ruse = []
urls_rusb = []
urls_ruse.extend(['https://www.londonstockexchange.com/' + scripts_ruse[i] for i in range(len(scripts_ruse))])
urls_rusb.extend(['https://www.londonstockexchange.com/' + scripts_rusb[i] for i in range(len(scripts_rusb))])

In [0]:
# Get the number of urls to determine the number of table rows:
nrows_ruse = len(urls_ruse)
nrows_rusb = len(urls_rusb)
# Create empty pandas dataframes with 0 to (nrows-1) rows and 0 to (4-1) columns:
table_ruse = pd.DataFrame(columns=range(0,4), index = range(0, nrows_ruse))
table_rusb = pd.DataFrame(columns=range(0,4), index = range(0, nrows_rusb))

In [0]:
# Fill the dataframe with RUSE data:
for row in range(nrows_ruse):
    resp = requests.get(urls_ruse[row])
    bs = BeautifulSoup(resp.text)
    script = bs.find('script', text = re.compile('DEALING DATE')).text

    date = re.search(re.compile(r'DEALING DATE: \d+\-\w{2,3}\-\d{4}'), script).group(0)
    table_ruse.iat[row,0] = date.split()[2]

    #nav = bs.find('p', text = re.compile('NAV PER SHARE')).text
    nav = re.search(re.compile(r'NAV PER SHARE: USD: \d+.\d+'), script).group(0)
    fnav = table_ruse.iat[row,1] = float(nav.split()[4])

    #shares = bs.find('p', text = re.compile('NUMBER OF SHARES')).text
    shares = re.search(re.compile(r'NUMBER OF SHARES IN ISSUE: \d+'), script).group(0)
    ishares = table_ruse.iat[row,2] = int(shares.split()[5])

    table_ruse.iat[row,3] = fnav*ishares

In [0]:
# Fill the dataframe with RUSB data:
for row in range(nrows_rusb):
    resp = requests.get(urls_rusb[row])
    bs = BeautifulSoup(resp.text)
    script = bs.find('script', text = re.compile('DEALING DATE')).text

    date = re.search(re.compile(r'DEALING DATE: \d+\-\w{2,3}\-\d{4}'), script).group(0)
    table_rusb.iat[row,0] = date.split()[2]

    nav = re.search(re.compile(r'NAV PER SHARE: USD: \d+.\d+'), script).group(0)
    fnav = table_rusb.iat[row,1] = float(nav.split()[4])

    shares = re.search(re.compile(r'NUMBER OF SHARES IN ISSUE: \d+'), script).group(0)
    ishares = table_rusb.iat[row,2] = int(shares.split()[5])

    table_rusb.iat[row,3] = fnav*ishares

In [0]:
# Rename the columns:
table_ruse.columns = table_rusb.columns = ['Date', 'NAV per share', 'Number of shares', 'Total NAV']

In [0]:
# Set the date column to datetime type:
table_ruse['Date'] = pd.to_datetime(table_ruse['Date'], dayfirst=True)
table_rusb['Date'] = pd.to_datetime(table_rusb['Date'], dayfirst=True)

In [0]:
# Do some type conversions:
table_ruse['NAV per share'] = pd.to_numeric(table_ruse['NAV per share'])
table_ruse['Number of shares'] = pd.to_numeric(table_ruse['Number of shares'])
table_ruse['Total NAV'] = pd.to_numeric(table_ruse['Total NAV'])
table_rusb['NAV per share'] = pd.to_numeric(table_rusb['NAV per share'])
table_rusb['Number of shares'] = pd.to_numeric(table_rusb['Number of shares'])
table_rusb['Total NAV'] = pd.to_numeric(table_rusb['Total NAV'])
# Change the index column:
table_ruse.index = pd.DatetimeIndex(table_ruse['Date'])
table_rusb.index = pd.DatetimeIndex(table_rusb['Date'])

In [0]:
# Get rid of the data column since it copies the index column now:
table_ruse = table_ruse.drop(columns=['Date']).dropna()
table_rusb = table_rusb.drop(columns=['Date']).dropna()

In [0]:
# Inverse the dataframes:
table_ruse = table_ruse.iloc[::-1]
table_rusb = table_rusb.iloc[::-1]

In [19]:
table_ruse.tail()

,NAV per share,Number of shares,Total NAV
Date,,,
2020-06-04,25.0777,189000,4739685.3
2020-06-05,25.6062,189000,4839571.8
2020-06-08,25.5886,189000,4836245.4
2020-06-09,25.5740,189000,4833486.0
2020-06-10,25.3441,189000,4790034.9


In [20]:
table_rusb.tail()

,NAV per share,Number of shares,Total NAV
Date,,,
2020-06-05,28.8359,465100,13411577.09
2020-06-08,28.8341,465100,13410739.91
2020-06-09,28.8354,465100,13411344.54
2020-06-10,28.8329,465100,13410181.79
2020-06-11,28.7582,465100,13375438.82
